# Mars Rovers
Author: Deirdre Corr
<br>
Date: 05/08/2019

In [ ]:
from pyllist import dllist
from pyllist import dllistnode
from pyllist import sllist
from enum import Enum
import re

### Create Enums for cardinal and action values

In [ ]:
class CardinalEnum(Enum):
    NORTH = "N"
    EAST = "E"
    SOUTH = "S"
    WEST = "W"
        
class ActionEnum(Enum):
    LEFT = "L"
    RIGHT = "R"
    MOVE = "M"

### Define a looped list class
- Extend the doubly linked list class
- Loop forward by returning the first node as the next node of the last
- Loop backwards by returning the last node as the previous node of the first

In [ ]:
class LoopedList(dllist):
    def __init__(self, list):
        dllist.__init__(self, list)
        
    def next_node(self, node):
        return self.first if node.next is None else node.next
        
    def prev_node(self, node):
        return self.last if node.prev is None else node.prev
    
    #dllist has no indexof(), so create method to iterate over list 
    #to find the node for a particular cardinal value
    def get_node(self, node_value):
        for i in super().iternodes():
            if i.value == node_value: return i
        raise ValueError('Error: node doesnt exist')

### Define robot class with x and y co-ordinates and it's cardinal direction

In [ ]:
class Robot():
    def __init__(self, x, y, direction):
        self.x = x
        self.y = y
        self.direction = direction
        
    def print_position(self):
        print(("{} {} "+ self.direction).format(self.x, self.y))
        
    #if action is to rotate right, use the next_node function
    #of loopedlist to get the cardinal direction to the right
    # if left, use prev_node to get the direction to the left
    def rotate(self, move_left, list):
        node = list.get_node(self.direction)
        if move_left:
            self.direction =  list.prev_node(node).value
        else:
            self.direction = list.next_node(node).value
        
    #increment/decrement the x/y value depending on which
    #direction the bot is facing
    def move(self):
        if self.direction == CardinalEnum.NORTH.value: self.y += 1
            
        elif self.direction == CardinalEnum.SOUTH.value: self.y -= 1
            
        elif self.direction == CardinalEnum.EAST.value: self.x += 1
            
        elif self.direction == CardinalEnum.WEST.value: self.x -= 1
            
        else: raise ValueError('Error: direction is not a cardinal value')

### Define the grid class with minimum and maximum x and y co-ordinates

In [ ]:
class Grid():
    def __init__(self, max_x, max_y):
        self.min_x = 0
        self.min_y = 0
        self.max_x = max_x
        self.max_y = max_y

### Define Navigation
- Loop through directions and call robot to rotate right, left or move
- Print the robot's new position

In [ ]:
def navigate(directions, robot, cardinal):
    for dir in directions:
        if dir == ActionEnum.LEFT.value: robot.rotate(True, cardinal)
            
        elif dir == ActionEnum.RIGHT.value: robot.rotate(False, cardinal)
            
        elif dir == ActionEnum.MOVE.value: robot.move() 
            
        else: raise ValueError('Error: only left, right and move directions permitted')
            
    robot.print_position()

### Add validation rules
To ensure the bot doesn't exit the plateau or crash into another bot

In [ ]:
def validate_position(this_robot, grid, squad):
    if this_robot.x > grid.max_x or this_robot.y > grid.max_y or this_robot.x < grid.min_x or this_robot.y < grid.min_y:
        raise PositionError('Error: outside of plateau boundaries')
    
    for member in squad:
        if member is not this_robot and (member.x == this_robot.x and member.y == this_robot.y):
            raise PositionError('Error: another robot is already in this position')
            
class PositionError(Exception):
    #Raise when robot should not move to the intended position
    pass

### Read Input
- Receive input in the form of a text file
- Normalise each line
- Ensure the text pattern matches allowed robot positions and actions

In [ ]:
def read_input():
    file = open("input.txt", "r")
    
    grid_input = normalise_input(file.readline())
    grid_coords = grid_input.split(' ')
    robots = {}

    for x in file:
        x = normalise_input(x)
        
        if bool(re.search(ROBOT_POSITION_REGEX, x)):
            path = normalise_input(file.readline())
            if not bool(re.search(ROBOT_ROUTE_REGEX, path)):
                raise ValueError('Error: only L,R,M characters permitted for route')
            robots[x] = path
            
    file.close
    return grid_coords, robots

def normalise_input(line):
    return line.upper().replace('\n','')

### Create robots and execute navigation
- Validate each robot's position
- Print final co-ordinates and orientation of each robot

In [ ]:
def complete_mission():
    grid_coords, robots = read_input()

    cardinal= LoopedList(["N","E","S","W"])
    grid = Grid(int(grid_coords[0]),int(grid_coords[1]))
    squad = sllist()
    
    print('Expected Output:')
    for robot_key, path in robots.items():
        params_list = robot_key.split(' ')
        
        robot = Robot(int(params_list[0]),int(params_list[1]),params_list[2])
        validate_position(robot, grid, squad)
        squad.append(robot)

        navigate(path, robot, cardinal)
        validate_position(robot, grid, squad)
        
        robot.print_position
        
ROBOT_POSITION_REGEX = '[0-9\s]{2}[NSEW]{1}'
ROBOT_ROUTE_REGEX = '^[LRM]+$'

### Execute mission

In [ ]:
complete_mission()